In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn import tree
import pickle
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import RandomizedSearchCV 

In [2]:
df = pd.read_csv('trainms.csv')
df.columns

Index(['s.no', 'Timestamp', 'Age', 'Gender', 'Country', 'state',
       'self_employed', 'family_history', 'treatment', 'work_interfere',
       'no_employees', 'remote_work', 'tech_company', 'benefits',
       'care_options', 'wellness_program', 'seek_help', 'anonymity', 'leave',
       'mental_health_consequence', 'phys_health_consequence', 'coworkers',
       'supervisor', 'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence', 'comments'],
      dtype='object')

In [3]:
temp = df.Gender
temp = temp.replace(to_replace=["A little about you","something kinda male?","Enby","fluid","Genderqueer","Trans-female","Nah","Androgyne","Agender","male leaning androgynous","Trans woman","Neuter","Female (trans)","queer","non-binary"],value="T")
temp = temp.replace(["Female","female","Cis Female","F","Woman","f","Femake","Female ","woman","cis-female/femme","Female (cis)"],'F')
temp = temp.replace(["M","Male","male","m","Male-ish","maile","Cis Male","Mal","Male ","Make","Guy (-ish) ^_^","Man","msle","Mail","cis male"],'M')
temp.unique()
df.Gender = temp

In [4]:
to_drop_columns = ['state','comments','s.no']+['Timestamp','no_employees','leave']+['supervisor','coworkers','Country','anonymity','self_employed','remote_work'] #+['Country','supervisor','coworkers','phys_health_interview','anonymity','self_employed','remote_work','tech_company','obs_consequence','phys_health_consequence','seek_help','Gender']

In [5]:
features = df.drop(columns=to_drop_columns)

In [6]:
target = features.pop('treatment')
colmns = features.columns
colmns.drop("Age",inplace=True)

TypeError: drop() got an unexpected keyword argument 'inplace'

In [ ]:
for column in colmns:
    features = pd.concat([features,pd.get_dummies(features[column], prefix=column)],axis=1)
    features.drop([column],axis=1, inplace=True)

age_data,_ = pd.factorize(df['Age'])
df["Age"] = age_data

In [ ]:
labels,_ = pd.factorize(target)

In [ ]:
features.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( features, labels, test_size = 0.3, random_state = 100)

In [ ]:
clf_gini = DecisionTreeClassifier(criterion = "gini",max_depth=3)
clf_gini.fit(X_train, y_train)

In [ ]:
from scipy.stats import randint 
param_dist = {"max_depth": [3, 4,5,6,7,None], 
              "max_features": randint(1, 9), 
              "min_samples_leaf": randint(1, 9), 
              "criterion": ["gini", "entropy"]} 
  
# Instantiating Decision Tree classifier 
tree = DecisionTreeClassifier() 
  
# Instantiating RandomizedSearchCV object 
tree_cv = RandomizedSearchCV(tree, param_dist, cv = 20) 
  
tree_cv.fit(X_train, y_train) 
  
# Print the tuned parameters and score 
print("Tuned Decision Tree Parameters: {}".format(tree_cv.best_params_)) 
print("Best score is {}".format(tree_cv.best_score_)) 

In [ ]:
gini_pred = clf_gini.predict(X_train)
accuracy_score(y_train,gini_pred)*100

In [ ]:
gini_pred = clf_gini.predict(X_test)
accuracy_score(y_test,gini_pred)*100